### Importación de librerías

In [ ]:
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Tratamiento de datos
# ==============================================================================
import pandas as pd
import numpy as np

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

# Liberías de optimización
# ==============================================================================
from pulp import *

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

### Importación de datos

In [ ]:
from google.colab import files 
files.upload()

In [304]:
# DEMANDA
demanda = pd.read_excel('/content/Prueba_Data.xlsx', sheet_name='DEMANDA_v2_list')
#demanda['DEMANDA'] = demanda['DEMANDA'].astype('int32')

# COSTOS FIJOS CTL
CTL = pd.read_excel('
                    .xlsx', sheet_name='CTL')
CTL = CTL[['CTL', 'COSTO_FIJO']]

Trasformación de datos

In [305]:
#Demanda en formato dict

origenes = demanda.ORIGEN.unique0.2111111111111111111111111111111111111()
destinos = demanda.DESTINO.unique()
DEMANDA_DICT = dict()
type_dict = 'records'

for origen in origenes:
  DEMANDA_DICT[origen] = {}
  for destino in destinos:
    valor = demanda[demanda.ORIGEN.isin([origen]) & demanda.DESTINO.isin([destino])]
    valor.reset_index(inplace=True, drop=True)
    DEMANDA_DICT[origen][destino] = valor.loc[0, 'DEMANDA']

print(DEMANDA_DICT)

{'EU': {'ANT': 162212, 'CAF': 2719, 'CUN': 151305, 'STR': 1077}, 'BRA': {'ANT': 90495, 'CAF': 2069, 'CUN': 158354, 'STR': 951}, 'USA': {'ANT': 702173, 'CAF': 3339, 'CUN': 330204, 'STR': 1799}, 'CA': {'ANT': 87633, 'CAF': 1300, 'CUN': 40380, 'STR': 33736}}


In [306]:
# Diccionario de trasbordos

trasbordos = CTL.CTL.unique()
CTL_DICT = dict()

for trasbordo in trasbordos:
  valor = CTL[CTL.CTL.isin([trasbordo])]
  valor.reset_index(inplace=True, drop=True)
  CTL_DICT[trasbordo] = valor.loc[0, 'COSTO_FIJO']

print(CTL_DICT)

{'PTB': 42417080, 'PLT': 47970275, 'DOR': 57785605, 'BCJ': 46863315, 'GRA': 47890870}


In [307]:
# COSTOS DE TRANSPORTE

costos = pd.read_excel('/content/Prueba_Data.xlsx', sheet_name='COSTOS')

# COSTOS DEL ORIGEN AL PUERTO
costos_puertos = costos[costos.Tipo1.isin(["ORIGEN"]) & costos.Tipo2.isin(["PUERTO"])]
costos_puertos = costos_puertos[['Nodo1_cod', 'Nodo2_cod', 'C_TOTAL']]
costos_puertos.rename(columns={'Nodo1_cod':'ORIGEN', 'Nodo2_cod':'PUERTO'}, inplace=True)
#display(costos_puertos)


# COSTOS DEL PUERTO AL PUNTO DE TRASBORDO
costos_ptm = costos[costos.Tipo1.isin(["PUERTO"]) & costos.Tipo2.isin(["TRASBORDO"])]
costos_ptm = costos_ptm[['Nodo1_cod', 'Nodo2_cod', 'Modo', 'C_TOTAL']]
costos_ptm.rename(columns={'Nodo1_cod':'PUERTO', 'Nodo2_cod':'TRASBORDO', 'Modo':'MODO'}, inplace=True)
costos_ptm.reset_index(inplace=True, drop=True)
costos_ptm['C_TOTAL'] = costos_ptm['C_TOTAL'].astype('int32')
#display(costos_ptm)


# COSTOS DEL PUNTO DE TRASBORDO AL DESTINO

costos_tj = costos[costos.Tipo1.isin(["TRASBORDO"]) & costos.Tipo2.isin(["DESTINO"])]
costos_tj = costos_tj[['Nodo1_cod', 'Nodo2_cod', 'C_TOTAL']]
costos_tj.rename(columns={'Nodo1_cod':'TRASBORDO', 'Nodo2_cod':'DESTINO'}, inplace=True)
costos_tj.reset_index(inplace=True, drop=True)
#costos_tj

In [308]:
# Costos del origen al puerto en formato dict

origenes = costos_puertos.ORIGEN.unique()
destinos = costos_puertos.PUERTO.unique()
CP_DICT = dict()
type_dict = 'records'

for origen in origenes:
  CP_DICT[origen] = {}
  for destino in destinos:
    valor = costos_puertos[costos_puertos.ORIGEN.isin([origen]) & costos_puertos.PUERTO.isin([destino])]
    valor.reset_index(inplace=True, drop=True)
    CP_DICT[origen][destino] = valor.loc[0, 'C_TOTAL']

print(CP_DICT)

{'CA': {'CTG': 563.88, 'BAQ': 628.32, 'STM': 796.18}, 'USA': {'CTG': 1010.92, 'BAQ': 1008.48, 'STM': 1237.04}, 'EU': {'CTG': 4672.8, 'BAQ': 4865.76, 'STM': 5271.36}, 'BRA': {'CTG': 3881.12, 'BAQ': 3960.0, 'STM': 4162.68}}


In [309]:
# Costos del puerto al trasbordo en formato dict

origenes = costos_ptm.PUERTO.unique()
destinos = costos_ptm.TRASBORDO.unique()
modos = costos_ptm.MODO.unique()
CPTM_DICT = dict()
type_dict = 'records'

for origen in origenes:
  CPTM_DICT[origen] = {}
  for destino in destinos:
    CPTM_DICT[origen][destino] = {}
    for modo in modos:
      valor = costos_ptm[costos_ptm.PUERTO.isin([origen]) & costos_ptm.TRASBORDO.isin([destino]) & costos_ptm.MODO.isin([modo])]
      valor.reset_index(inplace=True, drop=True)
      CPTM_DICT[origen][destino][modo] = valor.loc[0, 'C_TOTAL']

print(CPTM_DICT)

{'CTG': {'PTB': {'CARRETERO': 1191, 'FLUVIAL': 691, 'FERREO': 530}, 'PLT': {'CARRETERO': 324, 'FLUVIAL': 189, 'FERREO': 1000000000}, 'DOR': {'CARRETERO': 1552, 'FLUVIAL': 823, 'FERREO': 631}, 'BCJ': {'CARRETERO': 1180, 'FLUVIAL': 594, 'FERREO': 494}, 'GRA': {'CARRETERO': 879, 'FLUVIAL': 453, 'FERREO': 335}}, 'BAQ': {'PTB': {'CARRETERO': 1251, 'FLUVIAL': 664, 'FERREO': 455}, 'PLT': {'CARRETERO': 392, 'FLUVIAL': 162, 'FERREO': 1000000000}, 'DOR': {'CARRETERO': 1456, 'FLUVIAL': 796, 'FERREO': 561}, 'BCJ': {'CARRETERO': 1080, 'FLUVIAL': 576, 'FERREO': 425}, 'GRA': {'CARRETERO': 786, 'FLUVIAL': 427, 'FERREO': 264}}, 'STM': {'PTB': {'CARRETERO': 1161, 'FLUVIAL': 708, 'FERREO': 412}, 'PLT': {'CARRETERO': 441, 'FLUVIAL': 206, 'FERREO': 10000}, 'DOR': {'CARRETERO': 1395, 'FLUVIAL': 840, 'FERREO': 514}, 'BCJ': {'CARRETERO': 1021, 'FLUVIAL': 620, 'FERREO': 380}, 'GRA': {'CARRETERO': 724, 'FLUVIAL': 473, 'FERREO': 220}}}


In [310]:
# Costos del punto de trasbordo al destino en formato dict

origenes = costos_tj.TRASBORDO.unique()
destinos = costos_tj.DESTINO.unique()
CTJ_DICT = dict()
type_dict = 'records'

for origen in origenes:
  CTJ_DICT[origen] = {}
  for destino in destinos:
    valor = costos_tj[costos_tj.TRASBORDO.isin([origen]) & costos_tj.DESTINO.isin([destino])]
    valor.reset_index(inplace=True, drop=True)
    CTJ_DICT[origen][destino] = valor.loc[0, 'C_TOTAL']

print(CTJ_DICT)

{'PTB': {'ANT': 340.38, 'CAF': 732.4, 'CUN': 615.74, 'STR': 382.41}, 'PLT': {'ANT': 1032.98, 'CAF': 1628.43, 'CUN': 1619.13, 'STR': 887.91}, 'DOR': {'ANT': 470.25, 'CAF': 439.53, 'CUN': 359.29, 'STR': 622.54}, 'BCJ': {'ANT': 546.3, 'CAF': 882.49, 'CUN': 730.21, 'STR': 235.32}, 'GRA': {'ANT': 869.28, 'CAF': 1182.06, 'CUN': 1038.78, 'STR': 359.29}}


In [312]:
# Datos del problema

#origenes = ["MEX", "PAN", "USA", "DEU", "ESP", "BRA"]  # Nodos de origen
origenes = ["EU", "BRA", "USA", "CA"]  # Nodos de origen
#destinos = ["MED", "MNZ", "PER", "IBG", "NEV", "BOG", "BUG"]  # Nodos de destino
destinos = ["ANT", "CAF", "CUN", "STR"]  # Nodos de destino
puertos = ["CTG", "BAQ", "STM"]  # Nodos puertos
trasbordos = ["PLT", "PTB", "DOR", "BCJ", "GRA"]  # Nodos puntos de trasbordo
modos = ["CARRETERO", "FLUVIAL", "FERREO"] # Modos de transporte

# DEFINICIÓN DE PARÁMETROS

demanda = DEMANDA_DICT

capacidad_puerto = {"CTG": 10000000,"BAQ": 10000000,"STM": 10000000}

capacidad_trasbordo = {"PLT": 10000000, 
                       "PTB": 10000000, 
                       "DOR": 10000000, 
                       "BCJ": 10000000, 
                       "GRA": 10000000}

costo_origen_puerto = CP_DICT

costo_puerto_trasbordo = CPTM_DICT

costo_trasbordo_destino = CTJ_DICT

costo_fijo_trasbordo = CTL_DICT



### Modelo de optimización para la localización de puntos de trasbordo

In [318]:
# Variables
#x = {(i, j, p, t, m): 0 for i in origenes for j in destinos for p in puertos for t in trasbordos for m in modos}
x = LpVariable.dicts('Flujo', (origenes, destinos, puertos, trasbordos, modos),0)

#x = LpVariable.dicts('x', (origenes, destinos, puertos, trasbordos, modos), 0, None, LpContinuous)

# Variable de decisión
#y = LpVariable.dicts('y', (trasbordos), 0, 1, LpBinary)

y = LpVariable.dicts('y', (trasbordos), lowBound = 0, upBound = 1, cat='Integer')

# Model
model = LpProblem("Localizacion de puntos de trasbordo", LpMinimize)
'''
# Objective function
model += lpSum(costo_origen_puerto[i][p]* x[i, j, p, t, m] + 
               costo_puerto_trasbordo[p][t][m]*x[i, j, p, t, m]  + 
               costo_trasbordo_destino[t][j]*x[i, j, p, t, m] + 
               costo_fijo_trasbordo[t]*y[t] for i in origenes for j in destinos for p in puertos for t in trasbordos for m in modos)

model += lpSum(costo_origen_puerto[i][p]* x[i][j][p][t][m] + 
               costo_puerto_trasbordo[p][t][m]*x[i][j][p][t][m] + 
               costo_trasbordo_destino[t][j]*x[i][j][p][t][m] + 
               costo_fijo_trasbordo[t]*y[t] for i in origenes for j in destinos for p in puertos for t in trasbordos for m in modos)
'''
CT = lpSum(costo_origen_puerto[i][p]* x[i][j][p][t][m] + 
           costo_puerto_trasbordo[p][t][m]*x[i][j][p][t][m] + 
           costo_trasbordo_destino[t][j]*x[i][j][p][t][m] +
           costo_fijo_trasbordo[t]*y[t] for i in origenes for j in destinos for p in puertos for t in trasbordos for m in modos)
#CF = lpSum(costo_fijo_trasbordo[t]*y[t] for t in trasbordos)

model += CT

# Constraints
'''
# Restricciones de capacidad en los puertos
for p in puertos:
    model += lpSum(x[i, j, p, t, m] for i in origenes for j in destinos for t in trasbordos for m in modos) <= capacidad_puerto[p]

# Restricciones de capacidad en los trasbordos
for t in trasbordos:
    model += lpSum(x[i, j, p, t, m] for i in origenes for j in destinos for p in puertos for m in modos) <= capacidad_trasbordo[t]

# Restricciones de demanda satisfecha
for i in origenes:
    for j in destinos:
        model += lpSum(x[i, j, p, t, m] for p in puertos for t in trasbordos for m in modos) == demanda[i][j]
'''


# Restricciones de capacidad en los puertos
for p in puertos:
    model += lpSum(x[i][j][p][t][m] for i in origenes for j in destinos for t in trasbordos for m in modos) <= capacidad_puerto[p]

# Restricciones de capacidad en los trasbordos
for t in trasbordos:
    model += lpSum(x[i][j][p][t][m] for i in origenes for j in destinos for p in puertos for m in modos) <= capacidad_trasbordo[t]*y[t]

# Restricciones de demanda satisfecha
for i in origenes:
    for j in destinos:
        model += lpSum(x[i][j][p][t][m] for p in puertos for t in trasbordos for m in modos) == demanda[i][j]

#for j in destinos:
model += lpSum(y[t] for t in trasbordos) == 2

# Solve
model.solve()

1

### Prueba NO MODIFICAR

In [ ]:
# Datos del problema # Parameters
origenes = ['O1', 'O2', 'O3']  # Nodos de origen
destinos = ['D1', 'D2', 'D3']  # Nodos de destino
puertos = ['P1', 'P2', 'P3']  # Nodos puertos
trasbordos = ['T1', 'T2', 'T3']  # Nodos puntos de trasbordo
modos = ['carretero', 'ferreo', 'fluvial']

demanda = {'O1': {'D1': 10, 'D2': 15, 'D3': 20},
           'O2': {'D1': 5, 'D2': 8, 'D3': 12},
           'O3': {'D1': 7, 'D2': 10, 'D3': 15}}

capacidad_puerto = {'P1': 50,'P2': 50,'P3': 50}

capacidad_trasbordo = {'T1': 10000,'T2': 10000, 'T3': 10000}

costo_origen_puerto = {'O1': {'P1': 10, 'P2': 15, 'P3': 12},
                       'O2': {'P1': 8, 'P2': 11, 'P3': 9},
                       'O3': {'P1': 12, 'P2': 10, 'P3': 14}}

costo_puerto_trasbordo = {
    'P1': {'T1': {'carretero': 6, 'ferreo': 7, 'fluvial': 8}, 'T2': {'carretero': 5, 'ferreo': 6, 'fluvial': 7},'T3': {'carretero': 5, 'ferreo': 6, 'fluvial': 7}},
    'P2': {'T1': {'carretero': 8, 'ferreo': 9, 'fluvial': 10}, 'T2': {'carretero': 7, 'ferreo': 8, 'fluvial': 9}, 'T3': {'carretero': 5, 'ferreo': 6, 'fluvial': 7}},
    'P3': {'T1': {'carretero': 9, 'ferreo': 10, 'fluvial': 11}, 'T2': {'carretero': 10, 'ferreo': 11, 'fluvial': 12}, 'T3': {'carretero': 5, 'ferreo': 6, 'fluvial': 7}}
}


costo_trasbordo_destino = {
    'T1': {'D1': 9, 'D2': 12, 'D3': 11},
    'T2': {'D1': 8, 'D2': 10, 'D3': 13},
    'T3': {'D1': 5, 'D2': 13, 'D3': 10}
}

costo_fijo_trasbordo = {
    'T1': 95,
    'T2': 100,
    'T3': 100
}

In [ ]:
# CONJUNTO DE NODOS

origenes = ["MEX", "PAN", "USA", "DEU", "ESP", "BRA"]  # Nodos de origen
destinos = ["MED", "MNZ", "PER", "IBG", "NEV", "BOG", "BUG"]  # Nodos de destino
puertos = ["CTG", "BAQ", "STM"]  # Nodos puertos
trasbordos = ["PLT", "PTB", "DOR", "BCJ", "GRA"]  # Nodos puntos de trasbordo
modos = ["CARRETERO", "FLUVIAL", "FERREO"] # Modos de transporte

# DEFINICIÓN DE PARÁMETROS

demanda = {'MEX': {'MED': 87632.238, 'MNZ': 342, 'PER': 40379, 'IBG': , 'NEV': 20, 'BOG': 20, 'BUG': 20},
           'PAN': {'MED': 10, 'MNZ': 15, 'PER': 20, 'IBG': 20, 'NEV': 20, 'BOG': 20, 'BUG': 20},
           'USA': {'MED': 10, 'MNZ': 15, 'PER': 20, 'IBG': 20, 'NEV': 20, 'BOG': 20, 'BUG': 20},
           'DEU': {'MED': 10, 'MNZ': 15, 'PER': 20, 'IBG': 20, 'NEV': 20, 'BOG': 20, 'BUG': 20},
           'ESP': {'MED': 10, 'MNZ': 15, 'PER': 20, 'IBG': 20, 'NEV': 20, 'BOG': 20, 'BUG': 20},
           'BRA': {'MED': 10, 'MNZ': 15, 'PER': 20, 'IBG': 20, 'NEV': 20, 'BOG': 20, 'BUG': 20}}

capacidad_puerto = {"CTG": 10000000,"BAQ": 10000000,"STM": 10000000}

capacidad_trasbordo = {"PLT": 10000000, 
                       "PTB": 10000000, 
                       "DOR": 10000000, 
                       "BCJ": 10000000, 
                       "GRA": 10000000}

costo_origen_puerto = {'MEX': {'CTG': 10, 'BAQ': 15, 'STM': 12},
                       'PAN': {'CTG': 8, 'BAQ': 11, 'STM': 9},
                       'USA': {'CTG': 12, 'BAQ': 10, 'STM': 14},
                       'DEU': {'CTG': 10, 'BAQ': 15, 'STM': 12},
                       'ESP': {'CTG': 8, 'BAQ': 11, 'STM': 9},
                       'BRA': {'CTG': 12, 'BAQ': 10, 'STM': 14}}

costo_puerto_trasbordo = {
    'CTG': {'PLT': {'carretero': 6, 'ferreo': 7, 'fluvial': 8}, 'PTB': {'carretero': 5, 'ferreo': 6, 'fluvial': 7},'DOR': {'carretero': 5, 'ferreo': 6, 'fluvial': 7}, 'BCJ': {'carretero': 5, 'ferreo': 6, 'fluvial': 7}, 'GRA': {'carretero': 5, 'ferreo': 6, 'fluvial': 7}},
    'BAQ': {'PLT': {'carretero': 8, 'ferreo': 9, 'fluvial': 10}, 'PTB': {'carretero': 7, 'ferreo': 8, 'fluvial': 9}, 'DOR': {'carretero': 5, 'ferreo': 6, 'fluvial': 7}}, 'BCJ': {'carretero': 5, 'ferreo': 6, 'fluvial': 7}, 'GRA': {'carretero': 5, 'ferreo': 6, 'fluvial': 7},
    'STM': {'PLT': {'carretero': 9, 'ferreo': 10, 'fluvial': 11}, 'PTB': {'carretero': 10, 'ferreo': 11, 'fluvial': 12}, 'DOR': {'carretero': 5, 'ferreo': 6, 'fluvial': 7}, 'BCJ': {'carretero': 5, 'ferreo': 6, 'fluvial': 7}, 'GRA': {'carretero': 5, 'ferreo': 6, 'fluvial': 7}}
}


costo_trasbordo_destino = {
    'PLT': {'MED': 10, 'MNZ': 15, 'PER': 20, 'IBG': 20, 'NEV': 20, 'BOG': 20, 'BUG': 20},
    'PTB': {'MED': 10, 'MNZ': 15, 'PER': 20, 'IBG': 20, 'NEV': 20, 'BOG': 20, 'BUG': 20},
    'DOR': {'MED': 10, 'MNZ': 15, 'PER': 20, 'IBG': 20, 'NEV': 20, 'BOG': 20, 'BUG': 20},
    'BCJ': {'MED': 10, 'MNZ': 15, 'PER': 20, 'IBG': 20, 'NEV': 20, 'BOG': 20, 'BUG': 20},
    'GRA': {'MED': 10, 'MNZ': 15, 'PER': 20, 'IBG': 20, 'NEV': 20, 'BOG': 20, 'BUG': 20}
}

costo_fijo_trasbordo = {"PLT": 10000000,
                        "PTB": 10000000, 
                        "DOR": 10000000, 
                        "BCJ": 10000000, 
                        "GRA": 10000000}

In [ ]:
# Variables
#x = {(i, j, p, t, m): 0 for i in origenes for j in destinos for p in puertos for t in trasbordos for m in modos}
x = LpVariable.dicts('Flujo', (origenes, destinos, puertos, trasbordos, modos),0)

#x = LpVariable.dicts('x', (origenes, destinos, puertos, trasbordos, modos), 0, None, LpContinuous)

# Variable de decisión
#y = LpVariable.dicts('y', (trasbordos), 0, 1, LpBinary)

y = LpVariable.dicts('y', (trasbordos), lowBound = 0, upBound = 1, cat='Integer')

# Model
model = LpProblem("Localizacion de puntos de trasbordo", LpMinimize)
'''
# Objective function
model += lpSum(costo_origen_puerto[i][p]* x[i, j, p, t, m] + 
               costo_puerto_trasbordo[p][t][m]*x[i, j, p, t, m]  + 
               costo_trasbordo_destino[t][j]*x[i, j, p, t, m] + 
               costo_fijo_trasbordo[t]*y[t] for i in origenes for j in destinos for p in puertos for t in trasbordos for m in modos)

model += lpSum(costo_origen_puerto[i][p]* x[i][j][p][t][m] + 
               costo_puerto_trasbordo[p][t][m]*x[i][j][p][t][m] + 
               costo_trasbordo_destino[t][j]*x[i][j][p][t][m] + 
               costo_fijo_trasbordo[t]*y[t] for i in origenes for j in destinos for p in puertos for t in trasbordos for m in modos)
'''
CT = lpSum(costo_origen_puerto[i][p]* x[i][j][p][t][m] + 
           costo_puerto_trasbordo[p][t][m]*x[i][j][p][t][m] + 
           costo_trasbordo_destino[t][j]*x[i][j][p][t][m] +
           costo_fijo_trasbordo[t]*y[t] for i in origenes for j in destinos for p in puertos for t in trasbordos for m in modos)
#CF = lpSum(costo_fijo_trasbordo[t]*y[t] for t in trasbordos)

model += CT

# Constraints
'''
# Restricciones de capacidad en los puertos
for p in puertos:
    model += lpSum(x[i, j, p, t, m] for i in origenes for j in destinos for t in trasbordos for m in modos) <= capacidad_puerto[p]

# Restricciones de capacidad en los trasbordos
for t in trasbordos:
    model += lpSum(x[i, j, p, t, m] for i in origenes for j in destinos for p in puertos for m in modos) <= capacidad_trasbordo[t]

# Restricciones de demanda satisfecha
for i in origenes:
    for j in destinos:
        model += lpSum(x[i, j, p, t, m] for p in puertos for t in trasbordos for m in modos) == demanda[i][j]
'''


# Restricciones de capacidad en los puertos
for p in puertos:
    model += lpSum(x[i][j][p][t][m] for i in origenes for j in destinos for t in trasbordos for m in modos) <= capacidad_puerto[p]

# Restricciones de capacidad en los trasbordos
for t in trasbordos:
    model += lpSum(x[i][j][p][t][m] for i in origenes for j in destinos for p in puertos for m in modos) <= capacidad_trasbordo[t]*y[t]

# Restricciones de demanda satisfecha
for i in origenes:
    for j in destinos:
        model += lpSum(x[i][j][p][t][m] for p in puertos for t in trasbordos for m in modos) == demanda[i][j]

#for j in destinos:
model += lpSum(y[t] for t in trasbordos) == 3

# Solve
model.solve()


### Solver

In [319]:
print("Status:", LpStatus[model.status])

Status: Optimal


In [320]:
valor = model.objective.value()
print (valor)

18101642036.28


In [321]:
# Print results
print("Costo total: ", value(model.objective))
print("-------------------------------------------------")
for t in trasbordos:
        print("Trasbordo: ", t, " Costo fijo: ", costo_fijo_trasbordo[t], " Seleccionado: ", y[t].value())
print("-------------------------------------------------")
for i in origenes:
    for j in destinos:
        for p in puertos:
            for t in trasbordos:
                for m in modos:
                  if x[i][j][p][t][m].value() > 0:
                    print("Origen: ", i, " Destino: ", j, " Puerto: ", p, " Trasbordo: ", t, " Modo de transporte: ", m, " Cantidad: ", x[i][j][p][t][m].value())


Costo total:  18101642036.28
-------------------------------------------------
Trasbordo:  PLT  Costo fijo:  47970275  Seleccionado:  0.0
Trasbordo:  PTB  Costo fijo:  42417080  Seleccionado:  1.0
Trasbordo:  DOR  Costo fijo:  57785605  Seleccionado:  0.0
Trasbordo:  BCJ  Costo fijo:  46863315  Seleccionado:  1.0
Trasbordo:  GRA  Costo fijo:  47890870  Seleccionado:  0.0
-------------------------------------------------
Origen:  EU  Destino:  ANT  Puerto:  CTG  Trasbordo:  PTB  Modo de transporte:  FERREO  Cantidad:  162212.0
Origen:  EU  Destino:  CAF  Puerto:  CTG  Trasbordo:  PTB  Modo de transporte:  FERREO  Cantidad:  2719.0
Origen:  EU  Destino:  CUN  Puerto:  CTG  Trasbordo:  PTB  Modo de transporte:  FERREO  Cantidad:  151305.0
Origen:  EU  Destino:  STR  Puerto:  CTG  Trasbordo:  BCJ  Modo de transporte:  FERREO  Cantidad:  1077.0
Origen:  BRA  Destino:  ANT  Puerto:  CTG  Trasbordo:  PTB  Modo de transporte:  FERREO  Cantidad:  90495.0
Origen:  BRA  Destino:  CAF  Puerto:  CT